Vinnie's First Guided Project

This project is about something unknown at this point.

The explore_data() function takes in 4 parameters: 
    dataset, which is expected to be a list of lists
    start and end, which are both expected to be integers and represent the starting and the ending indices of a slice from the data set
    rows_and_columns, which is expected to be a Boolean and has False as a default argument
    
Slices the data set using dataset[start:end]

Loops through the slice, and for each iteration, prints a row and adds a new line after the row using print('\n').
    The \n in print('\n') is a special character that won't be printed. Instead, the \n character adds a new line, and we use print('\n') to add some blank space between rows. 

Prints the number of rows and columns if rows_and_columns is True. 
    dataset shouldn't have a header row, otherwise the function will print the wrong number of rows (one more row compared to the actual length)

In [ ]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row
        
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
        

Opens the Google Play and iOS Datasets

In [ ]:
from csv import reader

## Google Play Dataset ##
opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android = android[1:]

## App Store data set ##
opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

Prints the first few rows of the Google Play dataset

In [ ]:
print(android_header)
print('\n')
explore_data(android, 0, 3, rows_and_columns=True)

Prints the first few rows of the iOS dataset

In [ ]:
print(ios_header)
print('\n')
explore_data(ios, 0, 3, rows_and_columns=True)

Removes the row 10472 from the Google Play dataset because it has a rating of 19, and the max rating for a GP app is 5. 

In [ ]:
print(len(android))
del android[10472]
print(len(android))

Loops through the GP dataset to see how many entries Instagram has. 

In [ ]:
for app in android:
    name = app[0]
    if name == 'Instagram':
        print(app)

Counts the number of cases where an app occurs more than once (1,181)

In [ ]:
duplicate_apps = []
unique_apps = []

for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
        
print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:15])

We don't want to count certain apps mroe than once when we analyze data, so we need to remove the duplicate entries and keep only one entry per app. 

The main difference between duplicate entries is the number of reviews. The different numbers show the data was collected at different times. We'll keep the rows that have the highest number of reviews because the higher the number of reviews, the more reliable the ratings. 

To do that we:
    - create a dictionary where each key is a unique app name, and the value is the highest number of reviews of that app
    - use the dictionary to create a new data set, which will have only one entry per app (and we only select the apps with the highest number of reviews)

In [ ]:
reviews_max = {}

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

Prints the length of the dictionary "reviews_max"

In [ ]:
print('Expected length:', len(android) - 1181)
print('Actual length:', len(reviews_max))

Start by initializing two empty lists, android_clean adn already_added. 

Loop thrgouh the android data set, and for every iteration:
    - isolate the name of the app and the number of reviews
    - add the current row (app) to the android_clean list, and the app name (name) to the already_added list if:
        - the number of reviews of the current app matches the number of reviews of the app as described in the reviews_max dictionary; and
        - the name of the app is not already in the already_added list. This is needed to account for those cases where the highest number of reviews of a duplicate app is the same for more than one entry (i.e. the Box app has 3 entries, and the number of reviews is the same). 

In [ ]:
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if (reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name)

Explores new data set

In [ ]:
explore_data(android_clean, 0, 3, True)

Prints examples of non-english apps from both data sets

In [ ]:
print(ios[813][1])
print(ios[6731][1])

print(android_clean[4412][0])
print(android_clean[7940][0])

Want to remove non-enlgish apps. One way to do this is to remove each app whose name contains a symbol that is not commonly used in the English text - which usually includes letters from the english alphabet, numbers composed of digits from 0 to 9, punctuation marks(., !, ?, ;, etc), and other symbols (+, *, /, etc)

All these characters that are specific to English texts are encoded using the ASCII standard. Each ASCII character has a corresponding number between 0 and 127 associated with it, and we can take advantage of that to build a function that checks an app name and tells us whether it contains non-ASCII characters.

use the built-in ord() function to find out the corresponding encoding number of each character. 

In [ ]:
def is_english(string):
    
    for character in string:
        if ord(character) > 127:
            return False
        
    return True

print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))

Some English apps use emojis or other symbols that fall outside of the ASCII range. 

In [ ]:
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

print(ord('™'))
print(ord('😜'))

To minimize above impact, update is_english to only remove the app if the name has more than 3 non-ASCII characters. 

Still not perfect, but seems good enough at this point. 

In [ ]:
def is_english(string):
    non_ascii = 0
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
            
    if non_ascii > 3:
        return False
    else:
        return True
    
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

Filter out non-english apps in both data sets using is_english

In [ ]:
android_english = []
ios_english = []

for app in android_clean:
    name = app[0]
    if is_english(name):
        android_english.append(app)
        
for app in ios:
    name = app[1]
    if is_english(name):
        ios_english.append(app)
        
explore_data(android_english, 0, 3, True)
print('\n')
explore_data(ios_english, 0, 3, True)

Isolate only the free apps for analysis

In [ ]:
android_final = []
ios_final = []

for app in android_english:
    price = app[7]
    if price == '0':
        android_final.append(app)
        
for app in ios_english:
    price = app[4]
    if price == '0.0':
        ios_final.append(app)
        
print(len(android_final))
print(len(ios_final))

Because our end goal is to add the app on both the App Store and Google Play, we need to find app profiles that are successful on both markets. 

Begin the analysis by getting a sense of the most common genres for each market. For this, we will build a frequency table for the prime_genre column of the App Store data sert, and the Genres and Category columns of the Google Play data set. 

freq_table - generate frequency tables that show percentages
display_table - display percentages in a descending order

In [ ]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
            
    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage
        
    return table_percentages

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

examine the frequency table for the prime_genre column of the App Store data set. 

Among the free English apps, more than a half are games. 

The general impression is that App Store (at least the part containing English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.) while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous does not also imply that they also have the greatest number of users - the demand might not be the same as the offer. 

In [ ]:
display_table(ios_final, -5)

examine the Genres and Category columns of the Google Play dataset

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). 

If we investigate further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids. 

In [ ]:
display_table(android_final, 1)

In [ ]:
display_table(android_final, -4)

One way to figure out what genres are the most popular is to calculate the average number of installs for each app genre. For the GP data set, we can find this info in the Installs column, but for the App Store data set this info is missing. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the rating_count_tot app. 

Calculate the avg number of user ratings per app genre on the App Store. 

In [ ]:
genres_ios = freq_table(ios_final, -5)

for genre in genres_ios:
    total = 0
    len_genre = 0
    for app in ios_final:
        genre_app = app[-5]
        if genre_app == genre:
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = total / len_genre
    print(genre, ':', avg_n_ratings)

On average, nav apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million use reviews together. 

In [ ]:
for app in ios_final:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5])

Have 74942 user ratings on avg, but it's the Bible and Dictionary.com which skew up the avg rating. 

In [ ]:
for app in ios_final:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5])

Install numbers are precise enough - most values are open-ended (100+, 1000+, etc)

In [ ]:
display_table(android_final, 5)

To perform computations, we'll need to convert each install number to float - this means we need to remove thecommas and the plus characters, otherwise the conversion will fail and raise an error. 

Do this in the loop below, where we also compute the avg number of installs for each genre (category). 

In [ ]:
categories_android = freq_table(android_final, 1)

for category in categories_android:
    total = 0
    len_category = 0
    for app in android_final:
        category_app = app[1]
        if category_app == category:
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total / len_category
    print(category, ':', avg_n_installs)

Communication apps have the most installs, but is heavily skewed up by a few apps that have over a 1 billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts)

In [ ]:
for app in android_final:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                     or app[5] == '500,000,000+'
                                     or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

If we removed the communication apps that have over 100 million installs, the avg would be reduced rougly ten times: 

In [ ]:
under_100_m = []

for app in android_final:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

Take a look at some of the apps from the Books and Reference genre and their number of installs:

In [ ]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

Seems there's still a small number of extremely popular apps that skew the avg:

In [ ]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                           or app[5] == '500,000,000+'
                                           or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Looks like there are only a few very popular apps, so this market still shows potential. Let's try to get some app ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1000000 and 100000000 downloads)

Seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dicitonaries, so it's probably not a good idea to build similar apps since there'll be some significant competition. 

The market is already full of libraries, so  we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc. 

In [ ]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                           or app[5] == '5,000,000+'
                                           or app[5] == '10,000,000+'
                                           or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets. 

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and App Store markets. The markets are already full of libraries, so we need to add soem special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc. 